## Working with Sinergise service and API (work in progress)

In [1]:
import sys
sys.path.append('../../python3/')

In [2]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

# requires installation of sentinelhub python package and utils tool

from sentinelhub.utils import BBoxSelector
from sentinelhub import SentinelHubRequest, SentinelHubDownloadClient, DataSource, \
    MimeType, DownloadRequest, CRS, BBox

In [3]:
import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import xarray as xr

### Python API area selector

In [4]:
#selector = BBoxSelector(BBox(bbox=[-2.3, 5.1, -2.2, 5.2], crs=CRS.WGS84), zoom=9, resolution=100)
selector = BBoxSelector(BBox(bbox=[-2,5.9,-1.9,6], crs=CRS.WGS84), zoom=9, resolution=100)
selector.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [5]:
selector.bbox.crs, selector.bbox.min_x, selector.bbox.min_y, selector.bbox.max_x, selector.bbox.max_y, selector.size, (selector.bbox.max_x-selector.bbox.min_x)/selector.size[0]

(CRS('3857'),
 -222638.98158654713,
 657948.8109251115,
 -211507.03250721976,
 669141.0570442454,
 (111, 111),
 100.28782954348985)

### Query through WPS

In [6]:
# use WFS to find out available scenes
# for Sentinel-1 in AWS, TYPENAMES is set to DSS3
# To find out options https://services.sentinel-hub.com/ogc/wfs/796cd957-ea45-43d4-a54c-9d60bee9f325?REQUEST=GetCapabilities 
#!!NoCRS!!, use default 3857

instance_id = "796cd957-ea45-43d4-a54c-9d60bee9f325"
bbox = selector.bbox.min_x, selector.bbox.min_y, selector.bbox.max_x, selector.bbox.max_y
time = ("2020-04-01", "2020-04-30")

def get_features(instance_id, bbox, time, max_per_page=100):
    feature_offset = 0
    query_params = dict(REQUEST='GetFeature',
                        MAXFEATURES=str(max_per_page),
                        FEATURE_OFFSET=str(feature_offset),
                        OUTPUTFORMAT='application/json',
                        TYPENAMES="DSS3")

    query_params.update(BBOX=f'{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}')
    query_params.update(TIME=f'{time[0]}/{time[1]}')

    all_features = []
    num_features = max_per_page
    while num_features == max_per_page:
        query_params.update(FEATURE_OFFSET=str(feature_offset))
        response = requests.get(f'http://services.sentinel-hub.com/ogc/wfs/{instance_id}', params=query_params)

        if not response.ok:
            response.raise_for_status()
            raise response.reason

        feature_collection = response.json()
        if feature_collection.get('type') != 'FeatureCollection' \
                or not isinstance(feature_collection.get('features'), list):
            raise response.reason

        features = feature_collection['features']
        all_features.extend(features)
        num_features = len(features)
        feature_offset += num_features
    
    return all_features

all_features = get_features(instance_id, bbox, time)

In [7]:
from datetime import datetime, timedelta

for feature in all_features:
    start = datetime.strptime(feature['properties']['date'],"%Y-%m-%d")
    time_start, time_end = f"{start.isoformat()}Z", f"{(start+timedelta(1)).isoformat()}Z"
    print(time_start, time_end)

2020-04-27T00:00:00Z 2020-04-28T00:00:00Z
2020-04-21T00:00:00Z 2020-04-22T00:00:00Z
2020-04-15T00:00:00Z 2020-04-16T00:00:00Z
2020-04-09T00:00:00Z 2020-04-10T00:00:00Z
2020-04-03T00:00:00Z 2020-04-04T00:00:00Z


### Processing API

In [8]:
client_id = 'bae27d91-122e-4343-9b90-a0ad6d4b91b2'
client_secret = open('../../sentinelhub').read().strip()

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id,
                          client_secret=client_secret)

In [9]:
# this is not yet working as I'd expect


# Checking the authentication and the given access token to the user
headers = {
    "Accept":"application/x-tar",
    "Authorization": f"Bearer {token['access_token']}"
}
#setting the body of the evalscript with processing function for calculating the average in dB units
evalscript = """
//VERSION=3

function setup() {
  return {
    input: ["VV"],
    output: {id:"default", bands: 1},
    mosaicking: "ORBIT"
    }
}

function evaluatePixel(samples) {
  return [samples.VV]
}

""".rstrip()

# Setting the bounding box and coordinate reference system 
data = {
    "input": {
        "bounds":{
            "bbox":[
                bbox[0],bbox[1],bbox[2],bbox[3]
            ],
            "properties":{
                "crs":"http://www.opengis.net/def/crs/EPSG/0/3857"
            }
        },
        #Query of the S1GRD dataset using filtering options 
        "data": [
            {
                "type":"S1GRD",
                "dataFilter":{
                    "timeRange":{
                        "from":time_start,
                        "to":time_end
                    },
                    "orbitDirection": "ASCENDING"
                },
                "processing":{
                    "orthorectify": "true",
                    "upsampling": "BILINEAR",
                    "downsampling": "BILINEAR"
                }
            }
        ]
    },
    #Setting the output properties and format 
    "output": {
        "width":selector.size[0],
        "height":selector.size[1],
        "response":[
            {
                "identifier": "default",
                "format":{
                    "type":"image/tiff"
                }
            },
       ]
    },
    "evalscript": evalscript
}

url = "https://services.sentinel-hub.com/api/v1/process" 
data_response = requests.post(url, json=data, headers=headers)
print(data_response)

<Response [200]>


In [10]:
tiff_name = 'test.tif'
if data_response.status_code == 200:
    with open(tiff_name, 'wb') as f:
        for chunk in data_response:
            f.write(chunk)

### Python API

In [11]:
# this is not yet working as I'd expect

evalscript = """
    //VERSION=3

function setup() {
  return {
    input: ["VV", "VH"],
    output: {id:"default", bands: 2},
    mosaicking: "ORBIT"
    }
}

    function evaluatePixel(sample) {
        return [sample.VV, sample.VH];
    }
"""

request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_source=DataSource.SENTINEL1_IW,
            time_interval=('2020-04-01T00:00:00', '2020-04-30T00:00:00'),
        )
    ],#processing={"orthorectify": "true"},
    responses=[
        SentinelHubRequest.output_response('default', MimeType.TIFF),
        SentinelHubRequest.output_response('userdata', MimeType.JSON)
    ],
    bbox=selector.bbox,
    size=selector.size,
    #config=config
    data_folder='./',
)

In [12]:
tar = request.get_data(save_data=True)[0]

In [13]:
tar['default.tif'].shape, tar['userdata.json']

((111, 111, 2), None)